In [2]:
from typing import Dict, Any, Optional, List, Union
from netCDF4 import Dataset
import numpy as np
import logging
from datetime import datetime
from pyproj import Transformer


def XYtoLonLat(x,y):
    transformer = Transformer.from_crs("EPSG:32755", "EPSG:4326", always_xy=True)
    lon,lat = transformer.transform(x,y)
    return lon,lat

class NCReader:
    """NetCDF 文件读取器类"""
    
    def __init__(self, file_path: str):
        """
        初始化 NetCDF 读取器
        
        Args:
            file_path (str): NetCDF 文件路径
        """
        self.file_path = file_path
        self.nc = None
        
    def __enter__(self):
        self.nc = Dataset(self.file_path, mode='r')
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.nc is not None:
            self.nc.close()
    
    def get_global_attributes(self) -> Dict[str, Any]:
        """获取全局属性"""
        return {key: getattr(self.nc, key) for key in self.nc.ncattrs()}
    
    def get_variables(self) -> List[str]:
        """获取所有变量名称"""
        return list(self.nc.variables.keys())
    
    def get_dimensions(self) -> Dict[str, int]:
        """获取所有维度信息"""
        return {dim: len(self.nc.dimensions[dim]) for dim in self.nc.dimensions}
    
    def get_variable_info(self, var_name: str) -> Dict[str, Any]:
        """
        获取变量的详细信息
        
        Args:
            var_name (str): 变量名称
            
        Returns:
            Dict: 变量信息字典
        """
        if var_name not in self.nc.variables:
            raise ValueError(f"Variable '{var_name}' not found")
            
        var = self.nc.variables[var_name]
        return {
            'dimensions': var.dimensions,
            'shape': var.shape,
            'dtype': var.dtype,
            'attributes': {key: getattr(var, key) for key in var.ncattrs()},
            'units': getattr(var, 'units', None),
            'long_name': getattr(var, 'long_name', None)
        }
    
    def get_variable_data(self, 
                         var_name: str, 
                         start: Optional[Union[int, List[int]]] = None,
                         count: Optional[Union[int, List[int]]] = None) -> np.ndarray:
        """
        获取变量数据
        
        Args:
            var_name (str): 变量名称
            start: 起始索引
            count: 读取数量
            
        Returns:
            np.ndarray: 变量数据
        """
        if var_name not in self.nc.variables:
            raise ValueError(f"Variable '{var_name}' not found")
            
        var = self.nc.variables[var_name]
        return var[start:count] if start is not None else var[:]
    
    def get_time_variable(self, time_var_name: str = 'time') -> List[datetime]:
        """
        获取时间变量并转换为 datetime 对象列表
        
        Args:
            time_var_name (str): 时间变量名称
            
        Returns:
            List[datetime]: 时间列表
        """
        if time_var_name not in self.nc.variables:
            raise ValueError(f"Time variable '{time_var_name}' not found")
            
        time_var = self.nc.variables[time_var_name]
        units = getattr(time_var, 'units', '')
        
        # 解析时间单位
        if 'since' in units.lower():
            base_time_str = units.split('since')[1].strip()
            base_time = datetime.strptime(base_time_str, '%Y-%m-%d %H:%M:%S')
            
            # 转换时间数组
            time_values = time_var[:]
            return [base_time + np.timedelta64(int(t), 's') for t in time_values]
        else:
            raise ValueError(f"Unsupported time units: {units}")

# 使用示例
def main():
    file_path = "../3DiSimulations/215545.nc"
    
    try:
        with NCReader(file_path) as nc:
            # 获取全局属性
            global_attrs = nc.get_global_attributes()
            print("Global attributes:", global_attrs)
            
            # 获取所有变量
            variables = nc.get_variables()
            print("Variables:", variables)
            
            # 获取维度信息
            dimensions = nc.get_dimensions()
            print("Dimensions:", dimensions)
            
            # 获取特定变量信息
            var_info = nc.get_variable_info("Mesh2DNode_id")
            print("Variable info:", var_info)
            
            # 获取变量数据
            data = nc.get_variable_data("Mesh2DNode_id")
            print("Data shape:", data.shape)
            
            # 获取时间数据
            times = nc.get_time_variable()
            print("Time range:", times[0], "to", times[-1])
            
    except Exception as e:
        logging.error(f"Error processing NetCDF file: {str(e)}")
        raise

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    main()

ERROR:root:Error processing NetCDF file: [Errno 2] No such file or directory: '../3DiSimulations/215545.nc'


FileNotFoundError: [Errno 2] No such file or directory: '../3DiSimulations/215545.nc'

In [1]:
import rasterio


def get_closest_node_level(file_path: str):
    try:
        with NCReader(file_path) as nc:
            node_id = nc.get_variable_data("Mesh2DNode_id")
            xcc = nc.get_variable_data("Mesh2DFace_xcc")
            ycc = nc.get_variable_data("Mesh2DFace_ycc")
            level = nc.get_variable_data("Mesh2D_s1")
            lon, lat = XYtoLonLat(xcc, ycc)

            # Coordinates of the gauge 410001
            gauge_lat = -35.10077
            gauge_lon = 147.36836

            # Calculate the distance between the gauge and each node
            distances = np.sqrt((lat - gauge_lat)**2 + (lon - gauge_lon)**2)

            # Find the index of the node with the minimum distance to the gauge
            closest_node_index = np.argmin(distances)

            # Get the node id of the closest node
            closest_node_id = node_id[closest_node_index]

            # Get the level data corresponding to the closest_node_id
            closest_node_level = level[:, closest_node_index]

            return closest_node_level

    except Exception as e:
        logging.error(f"Error processing NetCDF file: {str(e)}")
        raise

def get_dem_value(file_path: str, lat: float = -35.10080647, lon: float = 147.367593175) -> float:
    try:
        with rasterio.open(file_path) as dem:
            # Convert the latitude and longitude to the DEM's coordinate system
            transformer = Transformer.from_crs("EPSG:4326", dem.crs, always_xy=True)
            x, y = transformer.transform(lon, lat)
            
            # Read the DEM value at the specified coordinates
            row, col = dem.index(x, y)
            dem_value = dem.read(1)[row, col]
            
            return dem_value

    except Exception as e:
        logging.error(f"Error processing DEM file: {str(e)}")
        raise

dem_path = "../SES_Fullstack_App/backend_python/data/3di_res/5m_dem.tif"
nc_path = "../3DiSimulations/215545.nc"
gauge_dem = get_dem_value(dem_path)
level = get_closest_node_level(nc_path)

print(f"Gauge DEM height: {gauge_dem}m")
print(f"Closest node level: {level}m")
#not good, the gauge coordinates seem to be on the ground not in the water
#need to check the satellite image in qgis to find the correct coordinates of the gauge


# -35.10077, 147.36836
# 410001 - MURRUMBIDGEE RIVER AT WAGGA WAGGA Lat:-35.10080647 Long:147.367593175 Elev:169.8m

NameError: name 'logging' is not defined